In [35]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr




In [36]:


from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter


from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings

In [37]:

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [38]:
MODEL = "gpt-4o-mini"


In [39]:
db_name="vector1"



In [40]:

folders=glob.glob("working/*")



In [42]:



folders = [folder for folder in glob.glob("working/*") if os.path.isdir(folder)]


documents = []


def add_metadata(doc, doc_type):
    doc.metadata = {"doc_type": doc_type}
    return doc


for folder in folders:
    doc_type = os.path.basename(folder)  # Using folder name as metadata
    try:
        
        loader = DirectoryLoader(folder, glob="*.txt", loader_cls=TextLoader, loader_kwargs={"encoding": "utf-8"})
        fdocs = loader.load()
        documents.extend([add_metadata(doc, doc_type) for doc in fdocs])
    except Exception as e:
        print(f"An error occurred while processing folder {folder}: {e}")


try:
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(documents)
    print(f"Number of chunks created: {len(chunks)}")
except Exception as e:
    print(f"An error occurred during text splitting: {e}")


Created a chunk of size 2777, which is longer than the specified 1000
Created a chunk of size 2743, which is longer than the specified 1000
Created a chunk of size 3048, which is longer than the specified 1000
Created a chunk of size 1788, which is longer than the specified 1000
Created a chunk of size 1880, which is longer than the specified 1000
Created a chunk of size 2462, which is longer than the specified 1000
Created a chunk of size 2797, which is longer than the specified 1000
Created a chunk of size 2483, which is longer than the specified 1000
Created a chunk of size 3113, which is longer than the specified 1000
Created a chunk of size 3148, which is longer than the specified 1000
Created a chunk of size 3106, which is longer than the specified 1000
Created a chunk of size 3066, which is longer than the specified 1000
Created a chunk of size 3117, which is longer than the specified 1000
Created a chunk of size 3178, which is longer than the specified 1000
Created a chunk of s

Number of chunks created: 910


In [44]:
textf={'encoding': 'utf-8'}


In [46]:
embed=OpenAIEmbeddings()

In [47]:
print(len(chunks))

910


In [48]:
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embed).delete_collection()

In [49]:
vectorstore=Chroma.from_documents(documents=chunks, embedding=embed, persist_directory=db_name)

In [50]:
vectorstore._collection.count()

910

In [51]:
##llm
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)
#memory
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever 
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

#connecting these 3
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)



In [52]:
def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

In [53]:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
